This script gathers perpetual future snapshots from kaiko

In [3]:
import requests
import pandas as pd
import datetime
import time

### raw

In [ ]:
def process_kaiko_raw(raw):
#trading data
    for i in range(len(raw)):
                    epoch = int(raw[i]['poll_timestamp']/1000)
                    time = datetime.datetime.utcfromtimestamp(epoch)
                
                    mid_price = raw[i]['mid_price']
                    spread = raw[i]['spread']
                    ask_slippage = raw[i]['ask_slippage']
                    bid_slippage = raw[i]['bid_slippage']
                    asks = raw[i]['asks']
                    bids = raw[i]['bids']

                    lst.append([epoch, time, mid_price, spread, ask_slippage, bid_slippage, asks, bids])
    return

In [ ]:
symbol = 'eth-usdt'
slippage = 1e6
lst = []

oderbook = 'snapshots'

#end_time = datetime.datetime.utcnow()
#start_time = end_time - datetime.timedelta(minutes=50)
#end_time_iso = end_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

start_time = datetime.datetime.utcfromtimestamp(1661126400) #22.08.2022
start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
counter = 0
cont = True
while cont:
    counter += 1
    header = {'X-Api-Key': '68cf96d421857181898149886cc12ddc'}
    query = f'https://eu.market-api.kaiko.io/v2/data/order_book_snapshots.v1/exchanges/binc/perpetual-future/{symbol}/{oderbook}/full?start_time={start_time_iso}&slippage={slippage}&sort=asc&page_size=100&slippage_ref=best'
    response = requests.get(query, headers=header)
    raw = response.json()
    #next_url = raw['next_url']
    raw = raw['data']
    
    print(counter, ': ', start_time_iso, 'to', raw[-1]['poll_date'])
    process_kaiko_raw(raw)
  
    start_time_iso = raw[-1]['poll_date']
    
    #end_time_iso = start_time_iso
    #start_time = start_time - datetime.timedelta(minutes=50)
    #start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    
    time.sleep(2)
    
    if int(raw[-1]['poll_timestamp']/1000) >= 1662987000: #today
        cont = False

In [ ]:
df = pd.DataFrame(lst, columns=['epoch', 'date', 'mid_price', 'spread', 'ask_slippage', 'bid_slippage', 'asks', 'bids'])
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)

df['ask_slippage'] = df['ask_slippage'].astype('float')
df['bid_slippage'] = df['bid_slippage'].astype('float')
df['spread'] = df['spread'].astype('float')

In [ ]:
#store in csv
df.to_csv(f'data/orderbook_eth_usdt.csv', index=False)

### aggregated

In [1]:
def process_kaiko_agg(raw):
#trading data
    for i in range(len(raw)):
                    epoch = int(raw[i]['poll_timestamp']/1000)
                    time = datetime.datetime.utcfromtimestamp(epoch)
                
                    mid_price = raw[i]['mid_price']
                    spread = raw[i]['spread']
                    ask_slippage = raw[i]['ask_slippage']
                    bid_slippage = raw[i]['bid_slippage']

                    lst.append([epoch, time, mid_price, spread, ask_slippage, bid_slippage])
    return

In [15]:
symbol = 'doge-usdt'
slippage = 1e6
lst = []

oderbook = 'ob_aggregations'
#interval = '1h'
interval = '1m'

start_time = datetime.datetime.utcfromtimestamp(1661126400) #22.08.2022
start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

In [ ]:
counter = 0
cont = True
while cont:
    counter += 1
    header = {'X-Api-Key': '68cf96d421857181898149886cc12ddc'}
    query = f'https://eu.market-api.kaiko.io/v2/data/order_book_snapshots.v1/exchanges/binc/perpetual-future/{symbol}/{oderbook}/full?start_time={start_time_iso}&interval={interval}&slippage={slippage}&sort=asc&page_size=100&slippage_ref=best'
    response = requests.get(query, headers=header)
    raw = response.json()
    raw = raw['data']

    process_kaiko_agg(raw)
    
    start_time_iso_old = start_time_iso
      
    start_time = datetime.datetime.utcfromtimestamp(raw[-1]['poll_timestamp']/1000)
    start_time_iso = start_time.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
        
    print(counter, ': ', start_time_iso_old, 'to', start_time_iso)
    
    time.sleep(2)
    
    if int(raw[-1]['poll_timestamp']/1000) >= 1663500000: #today
        cont = False

1 :  2022-08-22T00:00:00.000000Z to 2022-08-22T01:39:00.000000Z
2 :  2022-08-22T01:39:00.000000Z to 2022-08-22T03:18:00.000000Z


In [ ]:
df = pd.DataFrame(lst, columns=['epoch', 'date', 'mid_price', 'spread', 'ask_slippage', 'bid_slippage'])
df.sort_values(by=['epoch'], ascending=True, inplace=True)
df = df.drop_duplicates(subset=['epoch'], keep='first')
df = df.reset_index(drop=True)

df['ask_slippage'] = df['ask_slippage'].astype('float')
df['bid_slippage'] = df['bid_slippage'].astype('float')
df['spread'] = df['spread'].astype('float')
df

In [ ]:
#store in csv
df.to_csv(f'data/orderbook_btc_usdt_agg_1m.csv', index=False)